In [ ]:
import batoid
import numpy as np
import ipyvolume as ipv
import matplotlib.pyplot as plt

In [ ]:
fiducial_telescope = batoid.Optic.fromYaml("LSST_r.yaml")

In [ ]:
def xfan(telescope, theta_x, theta_y, nx):
    return batoid.RayVector.asGrid(telescope, wavelength=625e-9, theta_x=theta_x, theta_y=theta_y, nx=nx, ny=1)
def yfan(telescope, theta_x, theta_y, ny):
    return batoid.RayVector.asGrid(telescope, wavelength=625e-9, theta_x=theta_x, theta_y=theta_y, ny=ny, nx=1)
def xyfan(telescope, theta_x, theta_y, n):
    rays1 = xfan(telescope, theta_x, theta_y, n)
    rays2 = yfan(telescope, theta_x, theta_y, n)
    return batoid.concatenateRayVectors([rays1, rays2])

In [ ]:
def plotlineipv(ipv, ctf, x, y, z):
    x, y, z = np.broadcast_arrays(x, y, z)
    x, y, z = ctf.applyForwardArray(x, y, z)
    ipv.plot(x, y, z, c='r')

In [ ]:
def draw3dFP(ipv, coordSys):
    # Just has to be approximately right...
    raftsize = 0.31527*2/5
    ctf = batoid.CoordTransform(coordSys, batoid.globalCoordSys)
    
    # top hline
    plotlineipv(ipv, ctf, np.array([-1.5, 1.5])*raftsize, 2.5*raftsize, 0)
    # middle hlines
    plotlineipv(ipv, ctf, np.array([-2.5, 2.5])*raftsize, 1.5*raftsize, 0)
    plotlineipv(ipv, ctf, np.array([-2.5, 2.5])*raftsize, 0.5*raftsize, 0)
    plotlineipv(ipv, ctf, np.array([-2.5, 2.5])*raftsize, -0.5*raftsize, 0)
    plotlineipv(ipv, ctf, np.array([-2.5, 2.5])*raftsize, -1.5*raftsize, 0)
    # bottom hline
    plotlineipv(ipv, ctf, np.array([-1.5, 1.5])*raftsize, -2.5*raftsize, 0)
    # left vline
    plotlineipv(ipv, ctf, -2.5*raftsize, np.array([-1.5, 1.5])*raftsize, 0)
    # middle vlines
    plotlineipv(ipv, ctf, -1.5*raftsize, np.array([-2.5, 2.5])*raftsize, 0)
    plotlineipv(ipv, ctf, -0.5*raftsize, np.array([-2.5, 2.5])*raftsize, 0)
    plotlineipv(ipv, ctf, 0.5*raftsize, np.array([-2.5, 2.5])*raftsize, 0)
    plotlineipv(ipv, ctf, 1.5*raftsize, np.array([-2.5, 2.5])*raftsize, 0)
    # right vline
    plotlineipv(ipv, ctf, 2.5*raftsize, np.array([-1.5, 1.5])*raftsize, 0)

In [ ]:
def plotlineax(ax, ctf, x, y):
    x, y, z = np.broadcast_arrays(x, y, 0)
    x, y, _ = ctf.applyForwardArray(x, y, z)
    ax.plot(x, y, c='r')

In [ ]:
def draw2dFP(ax, coordSys):
    # Just has to be approximately right...
    raftsize = 0.31527*2/5
    ctf = batoid.CoordTransform(coordSys, batoid.globalCoordSys)
    # top hline
    plotlineax(ax, ctf, np.array([-1.5, 1.5])*raftsize, 2.5*raftsize)
    # middle hlines
    plotlineax(ax, ctf, np.array([-2.5, 2.5])*raftsize, 1.5*raftsize)
    plotlineax(ax, ctf, np.array([-2.5, 2.5])*raftsize, 0.5*raftsize)
    plotlineax(ax, ctf, np.array([-2.5, 2.5])*raftsize, -0.5*raftsize)
    plotlineax(ax, ctf, np.array([-2.5, 2.5])*raftsize, -1.5*raftsize)
    # bottom hline
    plotlineax(ax, ctf, np.array([-1.5, 1.5])*raftsize, -2.5*raftsize)
    # left vline
    plotlineax(ax, ctf, -2.5*raftsize, np.array([-1.5, 1.5])*raftsize)
    # middle vlines
    plotlineax(ax, ctf, -1.5*raftsize, np.array([-2.5, 2.5])*raftsize)
    plotlineax(ax, ctf, -0.5*raftsize, np.array([-2.5, 2.5])*raftsize)
    plotlineax(ax, ctf, 0.5*raftsize, np.array([-2.5, 2.5])*raftsize)
    plotlineax(ax, ctf, 1.5*raftsize, np.array([-2.5, 2.5])*raftsize)
    # right vline
    plotlineax(ax, ctf, 2.5*raftsize, np.array([-1.5, 1.5])*raftsize)    

In [ ]:
def doit(theta_x, theta_y, rot=0.0):
    telescope = fiducial_telescope.withLocallyRotatedOptic("LSSTCamera", batoid.RotZ(np.deg2rad(rot)))
    rays = xyfan(telescope, np.deg2rad(theta_x), np.deg2rad(theta_y), 30)
#     rays = xfan(telescope, np.deg2rad(theta_x), np.deg2rad(theta_y), 30)
    traceFull = telescope.traceFull(rays)
    zernike = batoid.zernike(
        telescope, np.deg2rad(theta_x), np.deg2rad(theta_y), 
        625e-9, jmax=6, nx=32
    )
    print(zernike[-2:])
    wavefront = batoid.wavefront(
        telescope, np.deg2rad(theta_x), np.deg2rad(theta_y),
        625e-9, nx=256
    )
    spotx, spoty = batoid.spot(
        telescope, np.deg2rad(theta_x), np.deg2rad(theta_y),
        625e-9, nx=256
    )
    ipv.figure(width=400, height=350)
    telescope.draw3d(ipv, color='black')
    batoid.drawTrace3d(ipv, traceFull, color='blue')
    ipv.xlim(-5, 5)
    ipv.ylim(-5, 5)
    ipv.zlim(-2, 8)
    draw3dFP(ipv, telescope['Detector'].coordSys)
    ipv.show()

    fpRays = traceFull['Detector']['out'].toCoordSys(batoid.globalCoordSys)
    w = ~fpRays.vignetted

    ph = np.linspace(0, 2*np.pi, 500)
    x, y = 0.31527*np.cos(ph), 0.31527*np.sin(ph)

    fig, axes = plt.subplots(figsize=(14, 3), nrows=1, ncols=5)
    ax1, ax2, ax3, ax4, ax5 = axes
    
    ax1.plot(x, y, c='b', lw=1)
    draw2dFP(ax1, telescope['Detector'].coordSys)
    ax1.scatter(fpRays.x[w], fpRays.y[w], c='b')
    ax1.set_xlim(-0.4, 0.4)
    ax1.set_ylim(-0.4, 0.4)
    ax1.axhline(0, lw=0.5, c='k')
    ax1.axvline(0, lw=0.5, c='k')
    ax1.set_title("view from utility trunk")
    ax1.set_xlabel("<- comes from left   x   comes from right ->")
    ax1.set_ylabel("<- higher alt    y    lower alt ->")
    ax1.set_aspect("equal")

    ax2.plot(x, y, c='b', lw=1)
    draw2dFP(ax2, telescope['Detector'].coordSys)
    ax2.scatter(fpRays.x[w], fpRays.y[w], c='b')
    ax2.set_xlim(0.4, -0.4)
    ax2.set_ylim(-0.4, 0.4)
    ax2.axhline(0, lw=0.5, c='k')
    ax2.axvline(0, lw=0.5, c='k')
    ax2.set_title("view from L3, tel aligned")
    ax2.set_xlabel("<- comes from right   x   comes from left ->")
    ax2.set_ylabel("<- higher alt    y    lower alt ->")
    ax2.set_aspect("equal")
    
    fpRays2 = traceFull['Detector']['out']
    w = ~fpRays2.vignetted

    ax3.plot(x, y, c='b', lw=1)
    draw2dFP(ax3, batoid.globalCoordSys)
    ax3.scatter(fpRays2.x[w], fpRays2.y[w], c='b')
    ax3.set_xlim(0.4, -0.4)
    ax3.set_ylim(-0.4, 0.4)
    ax3.axhline(0, lw=0.5, c='k')
    ax3.axvline(0, lw=0.5, c='k')
    ax3.set_title("view from L3, cam aligned")
    ax3.set_aspect("equal")
    
    ax4.scatter(-spotx, spoty, s=0.1, alpha=0.1)
    ax4.set_xlim(-1e-5, 1e-5)
    ax4.set_ylim(-1e-5, 1e-5)
    ax4.xaxis.set_ticks([])
    ax4.yaxis.set_ticks([])
    ax4.set_title("view from L3, cam aligned")
    ax4.set_aspect("equal")
    
    ax5.imshow(wavefront.array)
    ax5.set_aspect("equal")
    ax5.xaxis.set_ticks([])
    ax5.yaxis.set_ticks([])
    ax5.set_title("entrance pupil aligned")

In [ ]:
doit(1.0, 1.0, 0.0)

In [ ]:
# this is either +30 degrees rotTelPos or -30 degrees rotTelPos.  But which one!?
doit(1.0, 1.0, 30)